In [1]:
! pip install Bio

   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   -------------- ------------------------- 1.0/2.8 MB 5.6 MB/s eta 0:00:01
   ---------------------- ----------------- 1.6/2.8 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 4.5 MB/s eta 0:00:00


In [2]:
import os
import time
from Bio import Entrez, SeqIO

### Cào và gán nhãn kháng khuẩn AMP 

In [ ]:
Entrez.email = "tenGmail@gmail.com"
Entrez.api_key = "24dc460f63bff149fde9f635b305f230fa09"  # nếu có

def load_existing_sequences(filepath):
    """Tải các chuỗi sequence đã có trong file FASTA"""
    existing_sequences = set()
    if os.path.exists(filepath):
        with open(filepath, 'r') as file:
            for record in SeqIO.parse(file, 'fasta'):
                existing_sequences.add(str(record.seq))  # Thêm chuỗi sequence vào set
    return existing_sequences

def fetch_amp_sequences(amp_count=2600000, output_file='amp_genbank.fasta'):
    existing_sequences = load_existing_sequences(output_file)
    print(f"Đã có sẵn {len(existing_sequences)} chuỗi trong {output_file}, sẽ bỏ qua những chuỗi này.")

    if len(existing_sequences) >= amp_count:
        print(f"File {output_file} đã có đủ {amp_count} bản ghi. Dừng việc tải thêm dữ liệu.")
        return

    # Từ khóa tìm kiếm
    search_term_amp = 'Để câu lệnh truy vấn cào dữ liệu vào đây'

    print("Đang tìm kiếm các bản ghi AMP...")
    handle = Entrez.esearch(db="protein", term=search_term_amp, usehistory="y", retmax=0)
    search_results = Entrez.read(handle)
    handle.close()

    total_count = int(search_results["Count"])
    webenv = search_results["WebEnv"]
    query_key = search_results["QueryKey"]

    print(f"Tìm thấy tổng cộng {total_count} bản ghi. Sẽ tiến hành tải theo từng nhóm.")

    batch_size = 5000
    total_new = 0

    with open(output_file, 'a') as fasta_file:
        for start in range(0, total_count, batch_size):
            print(f"Fetching batch {start} - {start + batch_size}...")
            try:
                fetch_handle = Entrez.efetch(
                    db="protein",
                    rettype="fasta",
                    retmode="text",
                    retstart=start,
                    retmax=batch_size,
                    webenv=webenv,
                    query_key=query_key
                )
                for record in SeqIO.parse(fetch_handle, "fasta"):
                    seq = str(record.seq)
                    if seq not in existing_sequences:
                        fasta_file.write(f">{record.id} AMP\n{seq}\n")
                        existing_sequences.add(seq)
                        total_new += 1
                fetch_handle.close()
            except Exception as e:
                print(f"Lỗi ở batch {start} - {start + batch_size}: {e}")
                time.sleep(5)
                continue

            time.sleep(0.4)  # để tránh bị chặn API

            # Nếu đã đủ số lượng mong muốn thì dừng
            if len(existing_sequences) >= amp_count:
                print(f"Đã đủ {amp_count} chuỗi. Dừng việc tải.")
                break

    print(f"Đã ghi thêm {total_new} AMP mới vào {output_file}.")

fetch_amp_sequences()

### Cào và gán nhãn không kháng khuẩn nAMP 

In [ ]:
Entrez.email = "tenGmail@gmail.com"
Entrez.api_key = "24dc460f63bff149fde9f635b305f230fa09"  # nếu có

def load_existing_sequences(filepath):
    """Tải các chuỗi sequence đã có trong file FASTA"""
    existing_sequences = set()
    if os.path.exists(filepath):
        with open(filepath, 'r') as file:
            for record in SeqIO.parse(file, 'fasta'):
                existing_sequences.add(str(record.seq))  # Thêm chuỗi sequence vào set
    return existing_sequences

def fetch_namp_sequences(namp_count=2600000, output_file='namp_genbank.fasta'):
    existing_sequences = load_existing_sequences(output_file)
    print(f"Đã có sẵn {len(existing_sequences)} chuỗi trong {output_file}, sẽ bỏ qua những chuỗi này.")

    if len(existing_sequences) >= namp_count:
        print(f"File {output_file} đã có đủ {namp_count} bản ghi. Dừng việc tải thêm dữ liệu.")
        return

    # Từ khóa tìm kiếm
    search_term_amp = 'Để câu lệnh truy vấn cào dữ liệu vào đây'

    print("Đang tìm kiếm các bản ghi nAMP...")
    handle = Entrez.esearch(db="protein", term=search_term_amp, usehistory="y", retmax=0)
    search_results = Entrez.read(handle)
    handle.close()

    total_count = int(search_results["Count"])
    webenv = search_results["WebEnv"]
    query_key = search_results["QueryKey"]

    print(f"Tìm thấy tổng cộng {total_count} bản ghi. Sẽ tiến hành tải theo từng nhóm.")

    batch_size = 5000
    total_new = 0

    with open(output_file, 'a') as fasta_file:
        for start in range(0, total_count, batch_size):
            print(f"Fetching batch {start} - {start + batch_size}...")
            try:
                fetch_handle = Entrez.efetch(
                    db="protein",
                    rettype="fasta",
                    retmode="text",
                    retstart=start,
                    retmax=batch_size,
                    webenv=webenv,
                    query_key=query_key
                )
                for record in SeqIO.parse(fetch_handle, "fasta"):
                    seq = str(record.seq)
                    if seq not in existing_sequences:
                        fasta_file.write(f">{record.id} nAMP\n{seq}\n")
                        existing_sequences.add(seq)
                        total_new += 1
                fetch_handle.close()
            except Exception as e:
                print(f"Lỗi ở batch {start} - {start + batch_size}: {e}")
                time.sleep(5)
                continue

            time.sleep(0.4)  # để tránh bị chặn API

            # Nếu đã đủ số lượng mong muốn thì dừng
            if len(existing_sequences) >= namp_count:
                print(f"Đã đủ {namp_count} chuỗi. Dừng việc tải.")
                break

    print(f"Đã ghi thêm {total_new} nAMP mới vào {output_file}.")

fetch_namp_sequences()